In [3]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlunsplit, urlencode
MAX_P = 1000

out_dir = 'newsYahoo'
os.makedirs(out_dir, exist_ok=True)

In [5]:
MAINPAGE_URL = 'mainpage_url.txt'

In [6]:
base_url = 'https://news.yahoo.co.jp/topics/top-picks'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
req = requests.get(base_url, headers=agent) 
soup = BeautifulSoup(req.text, 'html.parser')



In [6]:
nav = soup.find('ul', attrs={'class': 'sc-kkwfeq bYUTVL'})
topics = nav.find_all('li')
for topic in topics:
    url_e = topic.find('a', href=True)
    if url_e is not None:
        url = base + url_e['href']
        with open(os.path.join(out_dir, MAINPAGE_URL), 'a+') as f:
            f.write(url + '\n')

# get all pages of each topic

In [7]:
def encode_page(url, p):
    param = urlencode({'page': p})
    return urlunsplit(("", "", url, param, ""))

In [40]:
def get_page_url(base_url, p):
#     print(base_url + "123")
    url = encode_page(base_url, p)
#     print(url)
    
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    atc_urls = []
    
    
    try:
        ul = soup.find('ul', attrs={'class': 'newsFeed_list'})

#         if ul is not None:

        links = ul.find_all('li', attrs={'class': 'newsFeed_item'})

    #     print(len(links))
        for link in links:
            atc_url = link.find('a', href=True)
            if atc_url is not None:
                atc_url = atc_url['href']
                atc_urls.append(atc_url)
    except:
        print(url)
    
        
    return atc_urls
    
    
#     print(links)
    
    

In [41]:
from tqdm import tqdm
with open(os.path.join(out_dir, MAINPAGE_URL), 'r') as f:
    base_urls = f.read().splitlines()
    
for base_url in base_urls:
    paths = []
    topic_name = base_url.split('/')[-1]
    
    out_topic_dir = os.path.join(out_dir, topic_name)
    os.makedirs(out_topic_dir, exist_ok=True)
    for p in tqdm(range(MAX_P)):
        urls = get_page_url(base_url, p+1)
#         print(urls)
#         input()
        if len(urls) == 0 or urls[-1] in paths:
            print(p, len(urls))
            break 
        paths.extend(urls)
        with open(os.path.join(out_topic_dir, '{}.txt'.format(topic_name)), 'a+') as f:
            for url in urls:
                f.write(url + '\n')
    
    

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

https://news.yahoo.co.jp/topics/top-picks?page=21
20 0


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

https://news.yahoo.co.jp/topics/domestic?page=5
4 0


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

...

https://news.yahoo.co.jp/topics/it?page=2
1 0


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

https://news.yahoo.co.jp/topics/science?page=2
1 0


  0%|▍                                                                                | 5/1000 [00:04<15:00,  1.10it/s]

https://news.yahoo.co.jp/topics/local?page=6
5 0


In [21]:
has_child_paths = os.listdir(out_dir)
has_child_paths = [path for path in has_child_paths if path.endswith('txt') and path != MAINPAGE_URL]
print(has_child_paths)

[]


In [22]:
def get_text(base_url, p):
    url = encode_page(base_url, p)
    text = ''
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    links = soup.find_all('p', attrs={'class': 'sc-fcdeBU gaNGyi yjSlinkDirectlink'}) 
    if links is None:
        return text
    for link in links:
        text += link.text.strip()
        
    links1 = soup.find_all('p', attrs={'class': 'sc-kZmsYB cBtiXl yjSlinkDirectlink'})  
    if links1 is None:
        return text
    for link in links1:
        text += link.text.strip()
    return text
    

In [25]:
def read_more(url):
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
#     print(soup)
    try:
        
        
        url1 = soup.find('p', attrs={'class': 'sc-bchHHi hDdzOI'}) 
        print(url1)
        if url1 is not None:
            url1 = url1.find('a')['href']
            return url1
        
        url1 = soup.find('p', attrs={'class': 'sc-kVyEtE fjDzrU'}) 
        print(url1)
        if url1 is not None:
            url1 = url1.find('a')['href']
            return url1
        
        
    except:
        print('no read more: '+url)
    return url

In [26]:
read_more('https://news.yahoo.co.jp/pickup/6393086')

None
<p class="sc-kVyEtE fjDzrU"><a class="sc-lewbHj ewGAmQ" data-ual-gotocontent="true" data-ylk="rsec:tpc_main;slk:headline;pos:2;" href="https://news.yahoo.co.jp/articles/8ad34a461a41263067760100e8e0129d80c96398">続きを読む</a></p>


'https://news.yahoo.co.jp/articles/8ad34a461a41263067760100e8e0129d80c96398'

In [25]:
def get_documents(url):
    document = {
        'topic': '',
        'time': '',
        'vender': '',
        'header': '',
        'body': '', 
        'url': ''
    }
    url = read_more(url)
#     print(url)
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    document['url'] = url
    #get topic
    try:
        document['topic'] = soup.find('li', attrs={'class': 'sc-ksYbfQ gzRlqt'}).find('a').text.strip()
    except:
        print('top: '+url)
        
    #get time
    try:
        document['time'] = soup.find('p', attrs={'class': 'sc-hBbWxd jAryWI'}).text.strip()
    except:
        print("time: "+url)
        
#     get header
    try:
        document['header'] = soup.find('h1', attrs={'class': 'sc-kcDeIU fSdkhd'}).text.strip()
    except:
        print("header: "+url)
        
#     get text

    try:
        text = ''
        for p in range(10):
            text1 = ''
            text1 = get_text(url, p+1)
            if len(text1) == 0:
                break
            text += text1
            
        
        document['body'] = text 
    except:
        print('text: '+url)
    
    return document

In [26]:
dirnames = os.listdir(out_dir)
dirnames = [name for name in dirnames if os.path.isdir(os.path.join(out_dir, name)) and 'ipynb' not in name]

print(dirnames)

['business', 'domestic', 'entertainment', 'it', 'local', 'science', 'sports', 'top-picks', 'world']


In [27]:
data_dir = 'data/news_yahoo'
os.makedirs(data_dir, exist_ok=True)

In [28]:
from tqdm import tqdm
import pandas as pd

i = 0

for name in dirnames:
    print('===========')
    print(name)
    paths = os.listdir(os.path.join(out_dir, name))
#     print(paths)
    txt_paths = [path for path in paths if os.path.isdir(os.path.join(out_dir, name, path))]
    
    print(txt_paths)
    
    if len(txt_paths) == 0:
        continue
        
    
    
    for c_path in txt_paths:
#         print('CHILD', c_path)
        c_a_paths = os.listdir(os.path.join(out_dir, name, c_path))
#         print(c_a_paths)
        for path in c_a_paths:
#             print(path)
            
            with open(os.path.join(out_dir, name, c_path, path), 'r') as f:
                article_urls = f.read().splitlines()
                
            for a_url in tqdm(article_urls):
                name_csv = a_url.split('/')[-2]
#                 print(name_csv)
                data = pd.DataFrame(columns=['topic', 'time', 'vender', 'header', 'body', 'url'])
                document = get_documents(a_url)
                data = data.append(document, ignore_index=True)
                data.to_csv(os.path.join(data_dir, name_csv + '.csv'))

  0%|                                                                                           | 0/65 [00:00<?, ?it/s]

business
['business.txt']
no read more: https://news.yahoo.co.jp/pickup/6393096


  2%|█▎                                                                                 | 1/65 [00:01<01:35,  1.49s/it]

time: https://news.yahoo.co.jp/pickup/6393096
header: https://news.yahoo.co.jp/pickup/6393096
text: https://news.yahoo.co.jp/pickup/6393096
no read more: https://news.yahoo.co.jp/pickup/6393086


  3%|██▌                                                                                | 2/65 [00:02<01:34,  1.50s/it]

...

  8%|██████▍                                                                            | 5/65 [00:07<01:35,  1.59s/it]

time: https://news.yahoo.co.jp/pickup/6393077
header: https://news.yahoo.co.jp/pickup/6393077
text: https://news.yahoo.co.jp/pickup/6393077
no read more: https://news.yahoo.co.jp/pickup/6393055


  8%|██████▍                                                                            | 5/65 [00:09<01:56,  1.93s/it]

time: https://news.yahoo.co.jp/pickup/6393055

KeyboardInterrupt: 

In [30]:
p = soup.find('p', attrs={'class': 'sc-kZmsYB cBtiXl yjSlinkDirectlink'})
p.text.strip()

'高齢者向けのワクチン接種の予約枠をめぐり、愛知県西尾市が、スギ薬局などを展開するドラッグストア大手「スギホールディングス（HD）」の杉浦広一会長（70）と昭子相談役（67）夫妻に優先的に便宜を図った問題で、スギHDはおわびのコメントを発表した。主な内容は次の通り。\n\n【画像】「会長が接種を楽しみにしています」と言われ…ワクチン便宜\u3000頭を下げる西尾市長ら\n\n\u3000会長杉浦広一および相談役昭子へのコロナワクチンの優先的接種を西尾市様に依頼したことにつきまして、ワクチン接種をお待ちの西尾市の方々はじめ、全国の皆さまにとって不快な行為であったこと、日夜尽力されている全国の行政の方々の努力に水を差す結果となってしまったことに深くおわび申し上げます。\n\n\u3000今回の案件に至った背景として、当社相談役が肺がんを患い大きな手術を経験しており、一日も早いワクチン接種をとおもんぱかった当社秘書が西尾市役所様にお問い合わせをさせていただいたことに端を発します。その使命感ゆえに何度かお問い合わせを繰り返ししたことについてご迷惑をおかけしたと考えております。\n\n\u3000また会長杉浦自身は過去にアナフィラキシーショックを経験しており、ワクチン接種を希望しておりません。\n\n\u3000このような事態を引き起こしたことを深く反省し、ちょうだいいたしました、多くのご意見・ご指摘を真摯（しんし）に受け止め、今後このようなことがなきよう努めてまいります。'

True
